In [2]:
import tensorflow as tf
from skimage import io
import numpy as np

In [76]:
image_file_path = r"F:\papers\FeaDistribution\spa\stego\0.2\0001.bmp"
img = io.imread(image_file_path)
img = img[1000:1100, 1000:1100, 0:1]
[height, width, channel] = np.shape(img)
# img = np.reshape(img, [height, width, channel])
img_new = np.zeros([2, height, width, channel], dtype=np.float32)
img_new[0] = img
img_new[1] = img

[[[132]
  [127]
  [119]
  ..., 
  [111]
  [114]
  [114]]

 [[134]
  [126]
  [135]
  ..., 
  [105]
  [105]
  [105]]

 [[136]
  [137]
  [141]
  ..., 
  [113]
  [114]
  [112]]

 ..., 
 [[123]
  [130]
  [136]
  ..., 
  [175]
  [192]
  [206]]

 [[127]
  [127]
  [124]
  ..., 
  [197]
  [199]
  [190]]

 [[127]
  [123]
  [123]
  ..., 
  [201]
  [194]
  [185]]]


In [77]:
img_tensor = tf.convert_to_tensor(img_new,
                                 dtype=tf.float32)
print(img_tensor.get_shape())

(2, 100, 100, 1)


In [61]:
downsampling_kernel1 = tf.constant(value=[1, 0, 0, 0],
                                   dtype=tf.float32,
                                   shape=[2, 2, 1, 1],
                                   name="downsampling_kernel1")

downsampling_kernel2 = tf.constant(value=[0, 1, 0, 0],
                                   dtype=tf.float32,
                                   shape=[2, 2, 1, 1],
                                   name="downsampling_kernel2")

downsampling_kernel3 = tf.constant(value=[0, 0, 1, 0],
                                   dtype=tf.float32,
                                   shape=[2, 2, 1, 1],
                                   name="downsampling_kernel3")

downsampling_kernel4 = tf.constant(value=[0, 0, 0, 1],
                                   dtype=tf.float32,
                                   shape=[2, 2, 1, 1],
                                   name="downsampling_kernel4")

In [78]:
x_stride, y_stride = 2, 2
padding = "VALID"
conv1 = tf.nn.conv2d(input=img_tensor,
                     filter=downsampling_kernel1,
                     strides=[1, x_stride, y_stride, 1],
                     padding=padding,
                     name="conv1")
conv2 = tf.nn.conv2d(input=img_tensor,
                     filter=downsampling_kernel2,
                     strides=[1, x_stride, y_stride, 1],
                     padding=padding,
                     name="conv2")
conv3 = tf.nn.conv2d(input=img_tensor,
                     filter=downsampling_kernel3,
                     strides=[1, x_stride, y_stride, 1],
                     padding=padding,
                     name="conv3")
conv4 = tf.nn.conv2d(input=img_tensor,
                     filter=downsampling_kernel4,
                     strides=[1, x_stride, y_stride, 1],
                     padding=padding,
                     name="conv4")
tf.concat([conv1, conv2, conv3, conv4], 3)


<tf.Tensor 'concat_8:0' shape=(2, 50, 50, 4) dtype=float32>

In [79]:
print(conv1.get_shape())

(2, 50, 50, 1)


In [80]:
def down_sampling(input_data, x_stride, y_stride, name, padding="VALID"):
    """
    get m * m(m is 2 in general) down sampling layer
    :param input_data: the input data tensor [batch_size, height, width, channels]
    :param x_stride: stride in X axis
    :param y_stride: stride in Y axis
    :param name: the name of the layer
    :param padding: the padding method, "SAME" | "VALID"
    :return:
        feature_map: 4-D tensor [number, height, width, channel]
    """
    conv1 = tf.nn.conv2d(input=input_data,
                         filter=downsampling_kernel1,
                         strides=[1, x_stride, y_stride, 1],
                         padding=padding,
                         name="conv1")

    conv2 = tf.nn.conv2d(input=input_data,
                         filter=downsampling_kernel1,
                         strides=[1, x_stride, y_stride, 1],
                         padding=padding,
                         name="conv2")

    conv3 = tf.nn.conv2d(input=input_data,
                         filter=downsampling_kernel1,
                         strides=[1, x_stride, y_stride, 1],
                         padding=padding,
                         name="conv3")

    conv4 = tf.nn.conv2d(input=input_data,
                         filter=downsampling_kernel1,
                         strides=[1, x_stride, y_stride, 1],
                         padding=padding,
                         name="conv4")

    conv_concat = tf.concat([conv1, conv2, conv3, conv4], 3, name=name)

    return conv_concat

In [81]:
conv = down_sampling(img_tensor, 2, 2, "conv", padding="VALID")
print(conv.get_shape())

(2, 50, 50, 4)


(2, 50, 50, 4)
[[[[ 132.]
   [ 119.]
   [ 126.]
   ..., 
   [ 128.]
   [ 117.]
   [ 114.]]

  [[ 136.]
   [ 141.]
   [ 144.]
   ..., 
   [ 110.]
   [ 116.]
   [ 114.]]

  [[ 112.]
   [ 122.]
   [ 132.]
   ..., 
   [ 109.]
   [ 110.]
   [ 112.]]

  ..., 
  [[ 128.]
   [ 116.]
   [ 127.]
   ..., 
   [ 224.]
   [ 213.]
   [ 185.]]

  [[ 124.]
   [ 135.]
   [ 123.]
   ..., 
   [ 220.]
   [ 179.]
   [ 184.]]

  [[ 127.]
   [ 124.]
   [ 128.]
   ..., 
   [ 220.]
   [ 190.]
   [ 199.]]]]


In [97]:
np.random.shuffle(img)

In [112]:
from glob import glob
def read_data(cover_files_path, stego_files_path, start_idx=None, end_idx=None, is_shuffle=True):
    """
    read file names from the storage
    :param cover_files_path: the folder name of cover files
    :param stego_files_path: the folder name of stego files
    :param start_idx: the start index
    :param end_idx: the end index
    :param is_shuffle: whether shuffle or not (default is True)
    :return:
        cover_data_list: list of cover data
        cover_label_list: list of cover label
        stego_data_list: list of stego data
        stego_label_list: list of stego label
    """
    cover_files_list = get_files_list(cover_files_path)         # data list of cover files
    stego_files_list = get_files_list(stego_files_path)         # data list of stego files
    sample_num = len(cover_files_list)                          # total pairs of samples
    cover_label_list = np.zeros(sample_num, np.int32)           # label list of cover files
    stego_label_list = np.ones(sample_num, np.int32)            # label list of stego files

    temp = np.array([cover_files_list, cover_label_list, stego_files_list, stego_label_list])
    print(temp[0:2])
    temp_t = temp.transpose()
    print(temp_t[0:2])

    if is_shuffle is True:
        np.random.shuffle(temp_t)

    if start_idx is not None and start_idx > sample_num:
        start_idx = 0
    if end_idx is not None and end_idx > sample_num:
        end_idx = sample_num + 1

    cover_data_list = list(temp_t[start_idx:end_idx, 0])
    cover_label_list = list(temp_t[start_idx:end_idx, 1])

    stego_data_list = list(temp_t[start_idx:end_idx, 2])
    stego_label_list = list(temp_t[start_idx:end_idx, 3])

    return cover_data_list, cover_label_list, stego_data_list, stego_label_list

def get_files_list(file_dir, file_type="txt", start_idx=None, end_idx=None):
    """
    get the files list
    :param file_dir: file directory
    :param file_type: type of files, "*" is to get all files in this folder
    :param start_idx: start index
    :param end_idx: end index
    :return:
        file_list: a list containing full file path
    """
    pattern = "/*." + file_type
    file_list = sorted(glob(file_dir + pattern))
    total_num = len(file_list)
    if type(start_idx) is int and start_idx > total_num:
        start_idx = None
    if type(end_idx) is int and end_idx > total_num:
        end_idx = None
    print(start_idx, end_idx)
    file_list = file_list[start_idx:end_idx]

    return file_list

def minibatches(cover_datas=None, cover_labels=None, stego_datas=None, stego_labels=None, batchsize=None):
    """
    read data batch by batch
    :param cover_datas: file name list (cover)
    :param cover_labels: label list(cover)
    :param stego_datas: file name list (stego)
    :param stego_labels: label list (stego)
    :param batchsize: batch size
    :return:
        yield datas and labels
    """
    for start_idx in range(0, len(cover_datas) - batchsize // 2 + 1, batchsize // 2):
        excerpt = slice(start_idx, start_idx + batchsize // 2)
        datas = cover_datas[excerpt]
        datas.extend(stego_datas[excerpt])
        labels = cover_labels[excerpt]
        labels.extend(stego_labels[excerpt])

        yield datas, labels

In [113]:
cover_files_path = r"E:\Myself\2.database\10.QMDCT\cover\cover_10s\128\test"
stego_files_path = r"E:\Myself\2.database\10.QMDCT\stego\EECS\EECS_B_128_W_2_H_7_ER_10\test"
a, b, c, d = read_data(cover_files_path, stego_files_path, start_idx=None, end_idx=None, is_shuffle=True)

for i in minibatches(a, b, c, d):
    print(i)
print("cover: ", a[0:5])
print("\n")
print("stego: ", c[0:5])

None None
None None
[[ 'E:\\Myself\\2.database\\10.QMDCT\\cover\\cover_10s\\128\\test\\wav10s_32001.txt'
  'E:\\Myself\\2.database\\10.QMDCT\\cover\\cover_10s\\128\\test\\wav10s_32002.txt'
  'E:\\Myself\\2.database\\10.QMDCT\\cover\\cover_10s\\128\\test\\wav10s_32003.txt'
  ...,
  'E:\\Myself\\2.database\\10.QMDCT\\cover\\cover_10s\\128\\test\\wav10s_33036.txt'
  'E:\\Myself\\2.database\\10.QMDCT\\cover\\cover_10s\\128\\test\\wav10s_33037.txt'
  'E:\\Myself\\2.database\\10.QMDCT\\cover\\cover_10s\\128\\test\\wav10s_33038.txt']
 ['0' '0' '0' ..., '0' '0' '0']]
[[ 'E:\\Myself\\2.database\\10.QMDCT\\cover\\cover_10s\\128\\test\\wav10s_32001.txt'
  '0'
  'E:\\Myself\\2.database\\10.QMDCT\\stego\\EECS\\EECS_B_128_W_2_H_7_ER_10\\test\\wav10s_32001.txt'
  '1']
 [ 'E:\\Myself\\2.database\\10.QMDCT\\cover\\cover_10s\\128\\test\\wav10s_32002.txt'
  '0'
  'E:\\Myself\\2.database\\10.QMDCT\\stego\\EECS\\EECS_B_128_W_2_H_7_ER_10\\test\\wav10s_32002.txt'
  '1']]
<generator object minibatches at 0x00